<a href="https://colab.research.google.com/github/ericsdata/colinsbeer/blob/main/src/WriteReviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 12.9 MB/s 
     |████████████████████████████████| 6.5 MB 69.9 MB/s 
     |████████████████████████████████| 77 kB 5.1 MB/s 
     |████████████████████████████████| 596 kB 62.2 MB/s 
     |████████████████████████████████| 895 kB 65.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
## Packages
import os
import pandas as pd
import torch
import pickle

from torch.utils.data import random_split, RandomSampler, SequentialSampler
from torch.utils.data import Dataset, DataLoader

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config,  AdamW, get_linear_schedule_with_warmup


In [3]:
## Class to hold beer text
class generative_BD(torch.utils.data.Dataset):
  '''Sequence text tokens
      This means it adds tags to start and end of texts

      Reads in text dataset, & tokenizes

      !!! NEED TO ADD PADDING TOKEN
  '''

  def __init__(self,text_list, tokenizer, text_tags, gpt2_type="gpt2", max_length=768):
    #self.text_list = text_list
    self.tokenizer = tokenizer
    self.input_ids = []
    self.attn_masks = []
    self.text_tags = [text_tags]

    ## In definitoin
    for txt in text_list:

        encodings_dict = tokenizer('<%s>'%(text_tags[0])+ txt + '<%s>'%(text_tags[1]), truncation=True, max_length=max_length, padding="max_length")

        self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
        self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

  def __getitem__(self, idx):
    return self.input_ids[idx], self.attn_masks[idx]

  def __len__(self):
    return len(self.input_ids)


In [4]:
### Data Read

with open("train_data", "rb") as fp:   # Unpickling
   revs = pickle.load(fp)

In [6]:
# Split into training and validation sets
train_size = int(0.8 * len(revs))
val_size = len(revs) - train_size

revs_train, revs_val = random_split(revs, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))


1,907 training samples
  477 validation samples


In [ ]:
### Init tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

## Add padding token
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

### Set up the data in datasets
train_dataset = generative_BD(text_list= revs_train, tokenizer = tokenizer, text_tags = ['CLS', 'SEP'])
val_dataset = generative_BD(text_list= revs_val, tokenizer = tokenizer, text_tags = ['CLS', 'SEP'])

In [ ]:
## Init paramters
batch_size = 2

In [ ]:
# Create the DataLoaders for our training and validation datasets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )